In [311]:
import numpy as np
import pandas as pd
import random
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv

from __future__ import print_function
import argparse

In [312]:
process_time = pd.read_csv("/workspaces/learnings/Data/data_actor_critic/process_time_matrix.csv")
work_order = pd.read_csv("/workspaces/learnings/Data/data_actor_critic/work_order.csv", names=["task_item","starttime","task","maxtime"])


In [313]:
process_time.head()

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,1,999999,49,41,999999,33,999999,999999,999999,999999,...,999999,21,29,999999,59,999999,10,46,999999,117
1,2,999999,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,178,999999,999999,999999,999999,999999,999999,999999,999999
2,3,999999,999999,999999,999999,168,6,999999,999999,999999,...,999999,999999,999999,65,999999,999999,999999,80,52,288
3,4,999999,999999,999999,999999,134,297,999999,999999,999999,...,999999,184,999999,153,999999,999999,999999,241,136,296
4,5,999999,7,620,999999,35,999999,999999,999999,113,...,999999,15,1058,999999,999999,999999,999999,13,28,6


In [314]:
work_order.head()

,task_item,starttime,task,maxtime
0,1,481,2,45
1,2,482,33,45
2,3,484,2,45
3,4,486,10,45
4,5,486,2,45


In [315]:
pd.DataFrame(work_order.groupby('task').task_item.count()).reset_index()

,task,task_item
0,1,1014
1,2,877
2,3,513
3,4,473
4,5,405
...,...,...
102,103,17
103,104,32
104,105,16
105,106,16


In [316]:
def v_wrap(np_array, dtype=np.float32):
    if np_array.dtype != dtype:
        np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)


def normalized_columns_initializer(weights, std=1.0):
    out = torch.randn(weights.size())
    out *= std / torch.sqrt(out.pow(2).sum(1, keepdim=True))
    return out


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        weight_shape = list(m.weight.data.size())
        fan_in = np.prod(weight_shape[1:4])
        fan_out = np.prod(weight_shape[2:4]) * weight_shape[0]
        w_bound = np.sqrt(6. / (fan_in + fan_out))
        m.weight.data.uniform_(-w_bound, w_bound)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        weight_shape = list(m.weight.data.size())
        fan_in = weight_shape[1]
        fan_out = weight_shape[0]
        w_bound = np.sqrt(6. / (fan_in + fan_out))
        m.weight.data.uniform_(-w_bound, w_bound)
        m.bias.data.fill_(0)


In [317]:
class Job_VM_env():
    path = '/workspaces/learnings/Data/data_actor_critic/'
    VM_task = pd.read_csv(path + 'process_time_matrix.csv',header=None).drop([0]).values
    task = pd.read_csv(path + 'work_order.csv',header=None).values
    
    def __init__(self):
        self.task_cluster = self.VM_task.shape[1]
        self.VM = self.VM_task.shape[0]
        self.task_num = self.task.shape[0]
        self.process_time = self.VM_task
        self.VM_status = np.repeat(0,self.VM) 
        self.VM_process_task = [[] for i in range(self.VM)]
        self.VM_process_time = [[] for i in range(self.VM)]
        self.task_waiting_time = [[] for i in range(self.VM)]
        self.left_task = self.task.shape[0]
        self.done = False
        self.total_time = 0  
        self.task_distribute_time = np.repeat(0,self.task.shape[0])
        self.total_task_process_time = np.repeat(0,self.task.shape[0])
        self.task_status = np.repeat(1,self.task.shape[0])  
        self.task_index = list(range(self.task.shape[0]))  
        self.timeindex = 0   
        self.state = np.vstack((self.task_status,self.task_distribute_time))
        self.state = self.state.reshape(self.state.shape[0],self.state.shape[1],1)
        self.done_task = [] 
        self.done_VM = [] 
        self.task_start_time = [] 
        self.state_dim = self.state.shape[0]
        self.action_dim = 2
        
        
    def reset(self):
        self.task_num = self.task.shape[0]
        self.VM_status = np.repeat(0,self.VM) 
        self.VM_process_task = [[] for i in range(self.VM)]
        self.VM_process_time = [[] for i in range(self.VM)]
        self.task_waiting_time = [[] for i in range(self.VM)]
        #self.left_task = self.task.shape[0]
        self.done = False
        self.total_time = 0  
        self.task_distribute_time = np.repeat(0,self.task.shape[0])
        self.total_task_process_time = np.repeat(0,self.task.shape[0])
        self.task_status = np.repeat(1,self.task.shape[0])  
        self.task_index = list(range(self.task.shape[0]))  
        #self.timeindex = 0  
        self.state = np.vstack((self.task_status,self.task_distribute_time))
        self.state = self.state.reshape(self.state.shape[0],self.state.shape[1],1)
        self.done_task = []
        self.done_VM = [] 
        self.task_start_time = [] 
        
        return self.state
        
    def step(self, action):
        
        task_id = np.random.choice(a=self.task_num, size=self.VM, replace=False, p=None)
        for i in task_id:
            if len(self.task_index) != 0:
                if i in self.task_index:
                    self.task_distribute_time[i] += 1
                    ## if more than 2, delete this task
                    #if self.task_distribute_time[i] >= 2:
                    #    del self.task_index[self.task_index.index(i)]
                else:
                    task_id[task_id.tolist().index(i)] = random.sample(self.task_index,1)[0]
            else:
                pass
        
        assert action.shape[0] == self.VM
        
        for i in range(self.VM):
            ## only process those tasks that are in task_index
            if task_id[i] in self.task_index:
                ## action = 0 indicates do not give tasks to the VM
                if action[i] == 0 or self.VM_status[i] == 3:
                    pass
                else:
                    self.VM_process_task[i].append(task_id[i])
                    self.VM_status[i] += 1
                    self.task_status[task_id[i]] = 0
                    self.VM_process_time[i].append(0)
                    # how much time a task wait before processing
                    self.task_waiting_time[i].append(self.timeindex)
                    # if VM could not handle the task, exit
                    self.total_task_process_time[task_id[i]] = self.process_time[i][self.task[task_id[i]][2]]
                
                delete_index = []
                for j in range(len(self.VM_process_time[i])):
                    if len(self.VM_process_task[i]) != 0:
                        if self.VM_process_time[i][j] == self.process_time[i][self.task[self.VM_process_task[i][j]][2]]:
                            # if task finished, workload of VM would decrease
                            self.VM_status[i] -= 1
                            self.done_VM.append(i)
                            if self.VM_process_task[i][j] not in self.done_task:
                                self.left_task -= 1
                            self.done_task.append(self.VM_process_task[i][j])
                            ## calculate when the task starts to be processed by subtracting the process time
                            self.task_start_time.append(self.task_waiting_time[i][j] + self.task[self.VM_process_task[i][j]][1])
                            delete_index.append(j)
                if len(delete_index) > 0:
                    if len(delete_index) > 1:
                        delete_index.sort(reverse = True)
                    for k in delete_index:
                        del self.VM_process_task[i][k]
                        del self.VM_process_time[i][k]
            ## calculate total time consumed
            self.total_time += sum(self.task_waiting_time[i]) + self.total_task_process_time[i].sum()
            self.VM_process_time[i] = [m + 1 for m in self.VM_process_time[i]]
        
        ## reward takes the minus of total time*0.001 and left task num
        #print(self.total_time)
        reward = 1 - self.left_task/self.task_num
        self.timeindex += 1
        
        ## update state info
        self.state = np.vstack((self.task_status,self.task_distribute_time))
        self.state = self.state.reshape(self.state.shape[0],self.state.shape[1],1)
        
        if self.left_task == 0:
            self.done = True
        #print(self.VM_status)
        #print(self.VM_process_task)
        #print(self.done_task)
        return self.state, reward, self.done, self.done_task, self.done_VM, self.task_start_time

    def update(self,delete_list):
        if len(delete_list) != 0:
            for i in delete_list:
                if i in self.task_index:
                    self.task_index.remove(i)
        else:
            pass

In [318]:

class ActorCritic(torch.nn.Module):
    def __init__(self, num_inputs, action_space):
        super(ActorCritic, self).__init__()
        self.conv1 = nn.Conv2d(num_inputs, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)

        self.lstm = nn.LSTMCell(32*553, 256)

        num_outputs = action_space
        self.critic_linear = nn.Linear(256, 1)
        self.actor_linear = nn.Linear(256, num_outputs)

        self.apply(weights_init)
        self.actor_linear.weight.data = normalized_columns_initializer(
            self.actor_linear.weight.data, 0.01)
        self.actor_linear.bias.data.fill_(0)
        self.critic_linear.weight.data = normalized_columns_initializer(
            self.critic_linear.weight.data, 1.0)
        self.critic_linear.bias.data.fill_(0)

        self.lstm.bias_ih.data.fill_(0)
        self.lstm.bias_hh.data.fill_(0)

        self.train()

    def forward(self, inputs):
        inputs, (hx, cx) = inputs
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = x.view(-1, 32*553)
        hx, cx = self.lstm(x, (hx, cx))
        x = hx

        return self.critic_linear(x), self.actor_linear(x), (hx, cx)
    
    def choose_action(self,inputs,action_dim):
        s, (hx, cx) = inputs
        value, logit, (hx, cx) = self.forward((s.unsqueeze(0),(hx, cx)))
        prob = F.softmax(logit, dim=-1)
        log_prob = F.log_softmax(logit, dim=-1)
        entropy = -(log_prob * prob).sum(1, keepdim=True)
        
        #action = prob.multinomial(num_samples=action_dim).detach()
        action=[]
        for i in range(action_dim):
            action.append(prob.multinomial(num_samples=1).detach()[0])
        action = torch.from_numpy(np.array(action,dtype = np.int64).reshape(1,133))
        return action, log_prob, entropy, value

In [319]:

def train():

    lr = 0.01
    gamma = 0.9
    gae_lambda = 1.00
    entropy_coef = 0.01
    value_loss_coef = 0.5
    max_grad_norm = 50
    seed= 2020
    num_steps = 100
    max_episode_length= 10
    episode = 10

    #torch.manual_seed(args.seed)

    env = Job_VM_env()
    
    model = ActorCritic(env.state_dim, env.action_dim)

    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()

    state = env.reset()
    state = v_wrap(state)
    done = True
    action_dim = env.VM

    episode_length = 0
    complete_tasks = []
    VM_complete_task = []
    complete_task_start_time = []
    update_list = []
    total_rewards =[]
    for episode in range(episode):
        
        if done:
            cx = torch.zeros(1, 256)
            hx = torch.zeros(1, 256)
        else:
            cx = cx.detach()
            hx = hx.detach()
        
        if len(complete_tasks) != 0:
            update_list = [n for m in complete_tasks for n in m]
            env.update(update_list)

        values = []
        log_probs = []
        rewards = []
        entropies = []

        for step in range(num_steps+1):
            episode_length += 1
            

            action, log_prob, entropy, value = model.choose_action((state, (hx,cx)),action_dim)
            #print("choosen action:", action, "with Prob:", log_prob, "value:", value)
            log_prob = log_prob.gather(1, action)[0]
            
            state, reward, done, done_task, done_VM, task_start_time = env.step(action.view(-1,).numpy())
            #print("state:", state, "reward at state action:", reward, "done_task", done_task, "done_VM", done_VM)

            done = done or episode_length >= max_episode_length
            ## reward shaping
            reward = np.round(max(min(reward, 1), -1),3)
            if episode_length % 20 == 0:
                print("reward:", reward)
                #print(done_task)

            if done:
                complete_tasks.append(done_task)
                VM_complete_task.append(done_VM)
                complete_task_start_time.append(task_start_time)
                #print('Complete these tasks with 100 iterations:')
                #print(complete_tasks)
                print('Current episode:',episode)
                episode_length = 0
                state = env.reset()
                
                final_output =[]
                for i in range(len(complete_tasks)):
                    for j in range(len(complete_tasks[i])):
                        output = ([complete_tasks[i][j]+1, VM_complete_task[i][j]+1, complete_task_start_time[i][j]])
                        final_output.append(output)

                final_output1 = pd.DataFrame(final_output)
                print("makespan:",final_output1[2].sum())
                final_output1.to_csv("output_file.csv")


            state = v_wrap(state)
            values.append(value)
            log_probs.append(log_prob)
            rewards.append(reward)
            entropies.append(entropy)
            
            if done:
                break
            
            print("total_reward:",rewards)

    
        

        R = torch.zeros(1, 1)
        if not done:
            value, _, _ = model((state.unsqueeze(0), (hx, cx)))
            R = value.detach()

        values.append(R)
        policy_loss = 0
        value_loss = 0
        gae = torch.zeros(1, 1)
        for i in reversed(range(len(rewards))):
            R = gamma * R + rewards[i]
            advantage = R - values[i]
            value_loss = value_loss + 0.5 * advantage.pow(2)

            # Generalized Advantage Estimation
            delta_t = rewards[i] + gamma * \
                values[i + 1] - values[i]
            gae = gae * gamma * gae_lambda + delta_t

            policy_loss = policy_loss - \
                log_probs[i] * gae.detach() - entropy_coef * entropies[i]

        optimizer.zero_grad()

        (policy_loss + value_loss_coef * value_loss).backward(torch.ones_like(policy_loss))
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        print("total loss (policy+value):", policy_loss.mean() + value_loss_coef * value_loss)
        print('parameters of model updated')
        
    

In [320]:
train()

total_reward: [0.0]
total_reward: [0.0, 0.0]
total_reward: [0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Current episode: 0
makespan: 1927
total loss (policy+value): tensor([[-0.1286]], grad_fn=<AddBackward0>)
parameters of model updated
total_reward: [0.0]
total_reward: [0.0, 0.0]
total_reward: [0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
total_reward: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Current episode: 1
makespan: 4298
total loss (policy+value): tensor([[54.1696]], grad_fn=<AddBackward0>)
parameters of model updated
total_reward: 